# Introducing Kotlin **StateFlows**

- `StateFlow` and `MutableStateFlow` are introduced.
- Their usage and related operators are discussed.
- `SharedFlow` is covered briefly.

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.*
import kotlinx.coroutines.channels.BufferOverflow

## Using `StateFlow`/`MutableStateFlow`

- `StateFlow` is a specialized type of Kotlin `Flow` designed to handle state-sharing scenarios. It is essentially a **hot stream** of data, meaning it always **holds the latest value** and emits it to new collectors.
- **Hot Stream**: Unlike regular flows that are **cold**, `StateFlow` is **hot** and continuously holds the latest value, which is immediately available to any collector.
- **Initial State**: It requires an initial value, ensuring that it always contains a state.
- **Mutability**: A `MutableStateFlow` is used to *update the state*, while `StateFlow` is the *read-only* version for consumers.
- **State Retention**: New subscribers *immediately receive the current state*, not just future updates.
- **No Backpressure**: `StateFlow` does not handle backpressure, which may be a consideration in high-demand data streams.
- **StateFlow Termination**: They emit data indefinitely unless their corotuine context closes; in that case they get closed automatically.

In [2]:
// Creating and using a simple 'StateFlow'

fun main() = runBlocking {

    /*
    'StateFlows' are usually created in a pair of mutable-immutable 'StateFlows'.
    The mutable state is used as backing property.
    */
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(10) { it ->
            delay(500)
            _uiState.value = it  // Updating the value of the 'StateFlow'
        }
    }

    val job2 = launch {
        /* 
        IMPORTANT: Here, we need to use 'uiState.take(10)' otherwise it will collect indefinitely.
        The operator 'take' works similar to the flows operator.
        
        We normally do not use 'take' because we want to get the updates indefinitely
        because the lifecycle of the coroutines are managed by the associated coroutine context 
        and the 'collect' and data emission will get cancelled properly whenerver 
        the context is cancelled.
        */
        uiState.take(10).collect { v: Int -> 
            // 'collect' indefinitely collects data.
            /*
            IMPORTANT: 'StateFlow' does not handle backpressure, it means that StateFlow keeps
            updating its value continuously, even if the downstream collector is slow or not
            ready to process it immediately. 
            
            It doesn't buffer or slow down the production of new data based 
            on how fast the downstream is consuming it.
            */
            delay(100) // Change it to 2000 to see what will happen
            println("Collected value is: $v")
        }
    }

    joinAll(job1, job2)
}

main()

Collected value is: -1
Collected value is: 0
Collected value is: 1
Collected value is: 2
Collected value is: 3
Collected value is: 4
Collected value is: 5
Collected value is: 6
Collected value is: 7
Collected value is: 8


## Updating and Collecting the Value of `StateFlow`s

- To update the value we can use different approaches:
  1. `[state].value = x` that directly assigns a new value `x` to the `StateFlow`. It is a simple value assignment that replaces the current value with the new one.
  2. `[state].update {...}` that allows you to modify the current state based on the existing state. It takes a lambda with the current state as input, performs some transformation, and then assigns the result as the new value.

- The `update` function in Kotlin's `MutableStateFlow` is **thread-safe**. It ensures that any updates to the state are done **atomically**, meaning it *avoids race conditions* when multiple coroutines or threads try to update the `StateFlow` simultaneously.

- If the state **changes** between the time it is read and when it is about to be updated (due to another thread or coroutine modifying it), **update will automatically retry** until it successfully applies the new value based on the current state.

In [3]:
// Using 'update {}' in 'MutabelStateFlow'

fun main() = runBlocking {
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)

            // Atomically update the value
            _uiState.update { v -> // Current value
                println("Value was updated.")
                x + v // Add x to current value and return
            } 
        }
    }

    val job2 = launch {
        uiState.take(10).collect { v: Int ->
            println("Collected value is: $v")
        }
    }

    joinAll(job1, job2)
}

main()

Collected value is: -1
Value was updated.
Value was updated.
Collected value is: 0
Value was updated.
Collected value is: 2
Value was updated.
Collected value is: 5
Value was updated.
Collected value is: 9
Value was updated.
Collected value is: 14
Value was updated.
Collected value is: 20
Value was updated.
Collected value is: 27
Value was updated.
Collected value is: 35
Value was updated.
Collected value is: 44


In [4]:
// Collecting the state value from multiple coroutines

fun main() = runBlocking {
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)

            // Atomically update the value
            _uiState.update { v -> // Current value
                println("Value was updated.")
                x + v // Adds values 
            } 
        }
    }

    val job2 = launch {
        uiState.take(5).collect { v: Int ->
            delay(100)
            println("Collected value from coroutine 1 is: $v")
        }
    }
    val job3 = launch {
        uiState.take(5).collect { v: Int ->
            delay(200)
            println("Collected value from coroutine 2 is: $v")
        }
    }

    joinAll(job1, job2, job3)
}

main()

Collected value from coroutine 1 is: -1
Collected value from coroutine 2 is: -1
Value was updated.
Value was updated.
Collected value from coroutine 1 is: 0
Collected value from coroutine 2 is: 0
Value was updated.
Collected value from coroutine 1 is: 2
Collected value from coroutine 2 is: 2
Value was updated.
Collected value from coroutine 1 is: 5
Collected value from coroutine 2 is: 5
Value was updated.
Collected value from coroutine 1 is: 9
Collected value from coroutine 2 is: 9
Value was updated.
Value was updated.
Value was updated.
Value was updated.
Value was updated.


## Some Useful StateFlow Operators

- `StateFlow` operators are almost similar to the `Flow` operatos.

In [5]:
// Using 'StateFlow' operators

fun main() = runBlocking {
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(6) { x ->
            delay(500)

            // Atomically update the value
            _uiState.update { v -> // Current value
                println("Value was updated.")
                x + v // Adds values 
            } 
        }
    }

    val job2 = launch {
        uiState.take(5).onStart {
            println("Collecting started.")
            emit(-2)
        }.onEach { v ->
            println("Collecting in progress: $v")
        }.transform { v ->
            val newValue = v+1
            println("Value is transformed to: $newValue")
            emit(newValue)
        }.map { v ->
            println("Mapping applied.")
            v + 1
        }.filter { v ->
            println("Filtering applied.")
            v > 0
        }.onCompletion {
            println("Collecting completed.")
        }.collect { v: Int ->
            println("Collected value is: $v")
        } // We can also use 'collect()' and 'launchIn()' operators. 
    }

    joinAll(job1, job2)
}

main()

Value is transformed to: -1
Mapping applied.
Filtering applied.
Value is transformed to: 0
Mapping applied.
Filtering applied.
Collected value is: 1
Value was updated.
Value was updated.
Value is transformed to: 1
Mapping applied.
Filtering applied.
Collected value is: 2
Value was updated.
Value is transformed to: 3
Mapping applied.
Filtering applied.
Collected value is: 4
Value was updated.
Value is transformed to: 6
Mapping applied.
Filtering applied.
Collected value is: 7
Value was updated.
Value is transformed to: 10
Mapping applied.
Filtering applied.
Collected value is: 11
Value was updated.


## Buffering on Consumer Site

- Kotlin `Flow` is **cold streams** and `StateFlow` is **hot**, so buffering is different when collecting `StateFlow`s.
- A buffer on consumer site, can speed up data **production-collection** in `Flow`s.
- However, `StateFlow` retains only the **latest** value (has an internal buffer with size 1), using a buffer **when collecting**, effectively acts as a mechanism **to prevent missing the recent data**. When combined with a buffer, it ensures that even if *the collector is slow, recent emissions are not lost*.
- The buffer's effectiveness will depend on its **size**; if the buffer is full, *older emissions may be discarded, but the most recent state is always retained*.

In [6]:
// An experiment without buffer

fun main() = runBlocking {
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)

            // Atomically update the value
            _uiState.update { v -> // Current value
                x
            } 
        }
    }

    val job2 = launch {
        uiState.take(5).collect { v: Int ->
            delay(1000)
            println("Collected value is: $v")
        } 
    }

    joinAll(job1, job2)
}

main()

Collected value is: -1
Collected value is: 0
Collected value is: 2
Collected value is: 4
Collected value is: 6


In [7]:
// Using buffer

fun main() = runBlocking {
    val _uiState: MutableStateFlow<Int> = MutableStateFlow(-1)
    val uiState: StateFlow<Int> = _uiState
    
    // Updating the mutable 'StateFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)

            // Atomically update the value
            _uiState.update { v -> // Current value
                x
            } 
        }
    }

    val job2 = launch {
        uiState.buffer(5).take(5).collect { v: Int ->
            delay(1000)
            println("Collected value is: $v")
        } 
    }

    joinAll(job1, job2)
}

main()

Collected value is: -1
Collected value is: 0
Collected value is: 1
Collected value is: 2
Collected value is: 3


## `SharedFlow`: A Combination of `Flow` and `StateFlow`

- A hot `Flow` that **shares** emitted values among all its collectors in a broadcast fashion, so that **all collectors** get **all emitted values**. It is not like channels when collectors consume data.
- `SharedFlow` is a **hot stream** Kotlin coroutine flow that allows multiple consumers to receive the same values emitted by **a single producer**. Unlike regular flows, which are consumed by **only one consumer**, `SharedFlows` can be shared among many.
- **Internal Buffering**: `SharedFlows` can have a buffer to store emitted values..
- **Replay**: `SharedFlows` can replay a certain number of **past emissions** to **new consumers**, providing a way to **catch up on missed events**. New subscribers (collectors) will get the data stored in the replay cache.
- **Share-In**: We can use `.shareIn()` to create a **hot stream** `SharedState` from a **cold** `Flow`.

In [8]:
// Examining replay cache of 'SharedFlow' 

fun main() = runBlocking {

    /*
    Creating a 'SharedFlow' with relpay size of 2. It caches last 2 emitted data for the new collcetors.
    By setting 'onBufferOverflow', we can tell the flow what to do when buffer is full.
    Options are:
        1. BufferOverflow.SUSPEND (suspends)
        2. BufferOverflow.DROP_OLDES (no suspension)
        3. BufferOverflow.DROP_LATEST (no suspension)
    */
    val _events = MutableSharedFlow<Int>(
        replay = 2, 
        onBufferOverflow = BufferOverflow.SUSPEND,  // Default is BufferOverflow.SUSPEND
        extraBufferCapacity = 1 // Additional buffer capacity beyond 'replay' size.
    ) // mutable shared flow
    val events = _events.asSharedFlow() // read-only shared flow
    // or
    //val events: SharedFlow<Int> = _events

    // Updating the mutable 'ShareFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)
            /*
            IMPORTANT: if we set 'onBufferOverflow = BufferOverflow.SUSPEND'
            and buffer fills, here 'emit' will suspend IF there exists any active suspending collector.
            Otherwise, it will CONTINUE emitting.
            */
            _events.emit(x) 
            println("Value $x emitted.")
        }
    }

    val job2 = launch {
        delay(3_000)
        /*
        Producer registers here after a delay. So at the moment it send collection request,
        it will use replay cache.
        */
        events.take(3).collect { v: Int ->
            println("Collected value is: $v")
        } 
    }

    joinAll(job1, job2)
    
}

main()

Value 0 emitted.
Value 1 emitted.
Value 2 emitted.
Value 3 emitted.
Value 4 emitted.
Collected value is: 3
Collected value is: 4
Value 5 emitted.
Collected value is: 5
Value 6 emitted.
Value 7 emitted.
Value 8 emitted.
Value 9 emitted.


In [9]:
// Examining internal buffer of 'SharedFlow' 

fun main() = runBlocking {

    /*
    Creating a 'SharedFlow' with relpay size of 2. It caches last 2 emitted data for the new collcetors.
    By setting 'onBufferOverflow', we can tell the flow what to do when buffer is full.
    Options are:
        1. BufferOverflow.SUSPEND
        2. BufferOverflow.DROP_OLDES
        3. BufferOverflow.DROP_LATEST
    */
    val _events = MutableSharedFlow<Int>(
        replay = 2, 
        onBufferOverflow = BufferOverflow.SUSPEND,  // Default is BufferOverflow.SUSPEND
        extraBufferCapacity = 1 // Additional buffer capacity beyond 'replay' size.
    ) 
    val events = _events.asSharedFlow() 


    // Updating the mutable 'ShareFlow'
    val job1 = launch {
        repeat(10) { x ->
            delay(500)
            /*
            IMPORTANT: if we set 'onBufferOverflow = BufferOverflow.SUSPEND'
            and buffer fills, here 'emit' will suspend IF there exists any active suspending collector.
            Otherwise, it will continue emitting.
            */
            _events.emit(x) 
            println("Value $x emitted.")
        }
    }

    val job2 = launch {
        
        // Consumer registers here immediately.
        events.take(3).collect { v: Int ->
            /*
            There is a delay in collection process so as 'onBufferOverflow' is set
            'BufferOverflow.SUSPEND' the producer gets suspended when its internal buffer fills.
            */
            delay(5_000)
            println("Collected value is: $v")
        } 
    }

    joinAll(job1, job2)
    
}

main()

Value 0 emitted.
Value 1 emitted.
Value 2 emitted.
Value 3 emitted.
Collected value is: 0
Value 4 emitted.
Collected value is: 1
Value 5 emitted.
Collected value is: 2
Value 6 emitted.
Value 7 emitted.
Value 8 emitted.
Value 9 emitted.


In [10]:
// Examining internal buffer of 'SharedFlow' 

fun main() = runBlocking {

    // Using 'shareIn' to create 'ShareState'
    val events = (1 until 10).asFlow().onEach { v ->
        delay(500)
        println("Value $v emitted.")
    }.shareIn(
        scope = this, // Share within the current coroutine scope
        replay = 2, // Replay the last 2 emitted values to new subscribers
        started = SharingStarted.Lazily  // Start sharing when the first subscriber subscribes [we have SharingStarted.Eagerly option]
    )


    val job = launch {
        events.take(3).collect { v: Int ->
            delay(1_000)
            println("Collected value is: $v")
        } 
    }

    joinAll(job)
    
}

main()

Value 1 emitted.
Value 2 emitted.
Collected value is: 1
Value 3 emitted.
Value 4 emitted.
Collected value is: 2
Value 5 emitted.
Value 6 emitted.
Collected value is: 3
Value 7 emitted.
Value 8 emitted.
Value 9 emitted.


## **DYE**: Do Your Experiment

In [11]:
// Ready for your codes